<a href="https://colab.research.google.com/github/GabrielLoiseau/tau-eval/blob/main/examples/Custom_Models_and_Metrics_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/GabrielLoiseau/tau-eval.git

Cloning into 'tau-eval'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 66 (delta 23), reused 56 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 44.44 KiB | 722.00 KiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
!cd tau-eval && pip install .

Processing /content/tau-eval
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [datasets]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.6.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# Implementing Custom Models and Metrics in Tau-Eval

This notebook guides users in creating their own anonymization models and evaluation metrics within the Tau-Eval framework. Tau-Eval is designed to be extensible, allowing researchers and developers to easily integrate and test custom components.


## Section 1: Custom Anonymization Models

Custom anonymization models in Tau-Eval should inherit from the `tau_eval.models.Anonymizer` base class. This class provides a standard interface for models, ensuring they can be seamlessly integrated into the evaluation pipeline.


In [ ]:
from tau_eval.models import Anonymizer
import random
import string
import wandb
wandb.init(mode="disabled")

# Define a simple custom model class
class NoisyTextModel(Anonymizer):
  def __init__(self, name: str = "NoisyText", noise_level: float = 0.1):
    self.name = name
    self.noise_level = noise_level

  def anonymize(self, text: str) -> str:
    """Adds random noise to text (deletion, case changes, insertion, swapping)."""
    result, i = [], 0
    while i < len(text):
        char = text[i]

        # Deletion: skip character
        if random.random() < self.noise_level * 0.3:
            i += 1
            continue

        # Case change for letters
        if char.isalpha() and random.random() < self.noise_level * 0.3:
            char = char.swapcase()

        result.append(char)

        # Insertion: add random letter
        if random.random() < self.noise_level * 0.2:
            result.append(random.choice(string.ascii_letters))

        # Swapping: swap current with next character
        if i < len(text) - 1 and random.random() < self.noise_level * 0.2:
            result.append(text[i + 1])
            i += 2
        else:
            i += 1

    return ''.join(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Instantiate the custom model
noisy_model = NoisyTextModel()

# Test the model's anonymize method
sample_text = "Hello, world!"
anonymized_text = noisy_model.anonymize(sample_text)
print(f"Original: {sample_text}")
print(f"Anonymized: {anonymized_text}")

Original: Hello, world!
Anonymized: Hello, world!


## Section 2: Custom Evaluation Metrics

Custom evaluation metrics in Tau-Eval are Python callables (e.g., functions) that adhere to a specific signature: `Callable[[str | list[str], str | list[str]], dict]`.

The callable takes two arguments:
1. `input_texts`: The original text(s) before anonymization.
2. `output_texts`: The anonymized text(s) produced by a model.
Both arguments can be a single string or a list of strings. The function should return a dictionary where keys are metric names (strings) and values are their corresponding scores (numerical values).

In [ ]:
import numpy as np

# Define a simple custom metric function
def jaccard_similarity_metric(input_texts: str | list[str],
                              output_texts: str | list[str]) -> dict[str, float]:
    if isinstance(input_texts, str):
      input_texts = [input_texts]
    if isinstance(output_texts, str):
      output_texts = [output_texts]

    scores = []
    for input_text, output_text in zip(input_texts, output_texts):
      # Simple word tokenization
      words_input = set(input_text.lower().split())
      words_output = set(output_text.lower().split())

      intersection = len(words_input.intersection(words_output))
      union = len(words_input.union(words_output))
      score = intersection / union if union > 0 else 0.0

      scores.append(score)

    return {'jaccard': np.mean(scores)}


# Test the custom metric
original_text = "This is the original text."
alternative_text = "This is the processed text with some changes."
anonymized_text = noisy_model.anonymize(original_text)

print(f"Jaccard Similarity: {jaccard_similarity_metric(original_text, alternative_text)}")

original_texts_list = [original_text, original_text]
processed_texts_list = [alternative_text, anonymized_text]
jaccard_score_dict_list = jaccard_similarity_metric(original_texts_list, processed_texts_list)
print(f"Jaccard Similarity for lists: {jaccard_score_dict_list}")

Jaccard Similarity: {'jaccard': np.float64(0.3)}
Jaccard Similarity for lists: {'jaccard': np.float64(0.36428571428571427)}


## Section 3: Using Custom Components in an Experiment

Once you have your custom model and metric, you can integrate them into a Tau-Eval `Experiment`. This involves:
1. Importing necessary classes: `Experiment`, `ExperimentConfig`.
2. Defining or importing a task. For this demonstration, we'll use a simple sentiment analysis task with a `tasksource` dataset.
3. Instantiating your custom model.
4. Creating lists of models and metrics, including your custom ones. Note that custom metric functions are passed directly.
5. Configuring the experiment with `ExperimentConfig`.
6. Running the experiment and observing the results.

In [ ]:
from tasknet import AutoTask
from tau_eval.tasks import IMDBAuthorshipClassification

sent = AutoTask("tweet_eval/sentiment", max_rows=1000, max_rows_eval=1000)
imdb = IMDBAuthorshipClassification(n_authors=10, max_rows=1000, max_rows_eval=1000)

README.md:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

mmlu.py:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/860 [00:00<?, ?B/s]

(…)-00000-of-00001-62894f3b39974716.parquet:   0%|          | 0.00/70.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61987 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/10000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from tau_eval import Experiment, ExperimentConfig

# Define the experiment configuration
models = [NoisyTextModel(name="NoisyText0.1",noise_level=0.1),
          NoisyTextModel(name="NoisyText0.3",noise_level=0.3),
          NoisyTextModel(name="NoisyText0.5",noise_level=0.5)]

metrics = [jaccard_similarity_metric, "meteor"]

tasks = [imdb, sent]

config = ExperimentConfig("full","answerdotai/ModernBERT-base",True,False)

e = Experiment(models, metrics, tasks, config)
e.run(output_dir="results.json")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

[06/20/25 14:02:43] DEBUG    Saved generated dataset                                                   ]8;id=288389;file:///usr/local/lib/python3.11/dist-packages/tau_eval/utils.py\utils.py]8;;\:]8;id=256787;file:///usr/local/lib/python3.11/dist-packages/tau_eval/utils.py#219\219]8;;\

DEBUG:tau_eval.logger:Saved generated dataset


                    INFO     Results saved                                                         ]8;id=776646;file:///usr/local/lib/python3.11/dist-packages/tau_eval/experiment.py\experiment.py]8;;\:]8;id=935518;file:///usr/local/lib/python3.11/dist-packages/tau_eval/experiment.py#92\92]8;;\

INFO:tau_eval.logger:Results saved


6. Print or show the results
The structure of 'results' might depend on the Experiment's implementation details, or how it's configured to return/display results.
Often, results are saved to files in 'output_dir' and/or returned.

In [ ]:
e.summary()['imdb_authorship_10_authors_0']

,Model Name,Accuracy,F1,meteor,jaccard
0,Original,0.973,0.971900,-,-
1,NoisyText0.1,0.684,0.640728,0.7355,0.5659
2,NoisyText0.3,0.341,0.196522,0.3954,0.2687
3,NoisyText0.5,0.263,0.143626,0.2402,0.1616


In [ ]:
e.summary()['tweet_eval/sentiment_1']

,Model Name,Accuracy,F1,meteor,jaccard
0,Original,0.697,0.687800,-,-
1,NoisyText0.1,0.639,0.610885,0.7737,0.597
2,NoisyText0.3,0.548,0.439357,0.4656,0.2806
3,NoisyText0.5,0.504,0.327534,0.2996,0.1571


Unsuprisingly, more noise leads to better privacy and worse utility!